 ## Check which version of spark that is installed:

In [20]:
import inspect
sc

<SparkContext master=yarn appName=pyspark-shell>

## Creating an RDD

In [32]:
# Create and RDD with students names and test scores to compute a GPA
# Use sc.parallelize to create RDD
# You can mix different objects with in a list
localTestScores =[['Aiden', 92], ['Aiden', 80], ['Aiden', 60],
                                ('Troy', 93), ('Troy', 77), ('Troy', 85),
                                ('Tyler', 75), ('Tyler', 80), ('Tyler', 90),
                                ]

RDDTestScorees = sc.parallelize(localTestScores, 2)
#Use the glom function on an RDD to see what each parition looks like
RDDTestScorees.glom().collect()



[[['Aiden', 92], ['Aiden', 80], ['Aiden', 60], ('Troy', 93)],
 [('Troy', 77), ('Troy', 85), ('Tyler', 75), ('Tyler', 80), ('Tyler', 90)]]

In [22]:
#This is a local python list that exists on the master i.e. PySpark driver
localTestScores

[['Aiden', 92],
 ['Aiden', 80],
 ['Aiden', 60],
 ('Troy', 93),
 ('Troy', 77),
 ('Troy', 85),
 ('Tyler', 75),
 ('Tyler', 80),
 ('Tyler', 90)]

In [24]:
#This is a RDD that exists distributed on the cluster
RDDTestScorees

ParallelCollectionRDD[79] at parallelize at PythonRDD.scala:480

In [25]:
#In memory lists are mutable but tuples are immuatable
#Change test score to 100 is possible
localTestScores[0][1] = 100
localTestScores

[['Aiden', 100],
 ['Aiden', 80],
 ['Aiden', 60],
 ('Troy', 93),
 ('Troy', 77),
 ('Troy', 85),
 ('Tyler', 75),
 ('Tyler', 80),
 ('Tyler', 90)]

In [26]:
#Fails to work because RDD are immutable and aren't random access
RDDTestScorees[0][1] = 100

TypeError: 'RDD' object does not support indexing

# Basic RDD

In [29]:
# Lets scale everyone's score down by 10% because there was cheating
# You have to call collect gather the results back into the driver to view it
RDDTestScorees.map(lambda entry: (entry[0], entry[1] * 0.9)).collect()

[('Aiden', 82.8),
 ('Aiden', 72.0),
 ('Aiden', 54.0),
 ('Troy', 83.7),
 ('Troy', 69.3),
 ('Troy', 76.5),
 ('Tyler', 67.5),
 ('Tyler', 72.0),
 ('Tyler', 81.0)]

In [ ]:
# You can compute the class GPA by
rddScaledScores = RDDTestScorees.map(lambda entry: (entry[1] * 0.9))
#Show pipeline maps i.e. the cluster tracks how the data was created
#rddScaledScores?

In [30]:
#gives you the class average
rddScaledScores.reduce(lambda s1,s2: s1 + s2) / rddScaledScores.count()

73.19999999999999

In [31]:
#To get the average per student then you need group the data by student
RDDTestScorees.groupByKey().collect()

[('Aiden', <pyspark.resultiterable.ResultIterable at 0x7f0319cafdd0>),
 ('Troy', <pyspark.resultiterable.ResultIterable at 0x7f0319cc2290>),
 ('Tyler', <pyspark.resultiterable.ResultIterable at 0x7f0319cc21d0>)]

In [33]:
import operator
# computing the averager per student is going to require doing an aggregateByKey on the RDD
# The data of a single student can be store in more than 1 parition i.e. exists on a different worker
# In this example Troy exists in the first and second parition
# We need to specifiy how to combine it properly
# You need to provide a starting zeroValue
starting_value = {'total_score' : 0,
                 'cnt_tests' : 0}

# You modifying the input argument accumulator
# this function gets applied to each element per partition
def seq_op(accumulator, test_score):
  accumulator['total_score'] = accumulator['total_score'] + test_score
  accumulator['cnt_tests'] = accumulator['cnt_tests'] + 1
  return accumulator

# combines the result across parition therefore it will be combining dictionaries
def comb_op(dic_p1, dic_p2):
  dic_p1['total_score'] = dic_p1['total_score'] + dic_p2['total_score'] 
  dic_p1['cnt_tests'] = dic_p1['cnt_tests'] +  dic_p2['cnt_tests'] 
  return dic_p1

per_student_stats = RDDTestScorees.aggregateByKey(starting_value, seq_op, comb_op)
per_student_stats.collect()


[('Aiden', {'cnt_tests': 3, 'total_score': 232}),
 ('Troy', {'cnt_tests': 3, 'total_score': 255}),
 ('Tyler', {'cnt_tests': 3, 'total_score': 245})]

In [34]:
per_student_stats.map(lambda student_stats:
                    (student_stats[0], float(student_stats[1]['total_score']) / student_stats[1]['cnt_tests'])).collect()

[('Aiden', 77.33333333333333), ('Troy', 85.0), ('Tyler', 81.66666666666667)]

# PySpark DataFrame

In [35]:
#Get access google cloud storage
import google.datalab.storage as storage
import pyspark.sql.functions as func
from pyspark.sql.types import *
from pyspark.sql import *
import pyspark.sql.window

bucket = storage.Bucket('mth9898-bucket')

#field, data_type, nullable
csv_schema = StructType([StructField("Date", DateType(), True),    
                        StructField("Open", DoubleType(), True),
                        StructField("High", DoubleType(), True),
                        StructField("Low", DoubleType(), True),
                        StructField("Close", DoubleType(), True),
                        StructField("Adj Close", DoubleType(), True),
                        StructField("Volume", LongType(), True)])

df_sp_500 = spark.read.csv(bucket.object('sp_500.csv').uri, header=True, schema=csv_schema)
df_sp_500.printSchema()


root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: long (nullable = true)



In [36]:
df_sp_500.show()

+----------+-----------+-----------+-----------+-----------+-----------+----------+
|      Date|       Open|       High|        Low|      Close|  Adj Close|    Volume|
+----------+-----------+-----------+-----------+-----------+-----------+----------+
|2013-02-11|1517.930054|1518.310059|1513.609985| 1517.01001| 1517.01001|2684100000|
|2013-02-12| 1517.01001|1522.290039|1515.609985|1519.430054|1519.430054|3414370000|
|2013-02-13|1519.430054|1524.689941|1515.930054|1520.329956|1520.329956|3385880000|
|2013-02-14|1520.329956|1523.140015| 1514.02002|1521.380005|1521.380005|3759740000|
|2013-02-15|1521.380005| 1524.23999|1514.140015|1519.790039|1519.790039|3838510000|
|2013-02-19|1519.790039|1530.939941|1519.790039|1530.939941|1530.939941|3748910000|
|2013-02-20|1530.939941|1530.939941|1511.410034|1511.949951|1511.949951|4240570000|
|2013-02-21|1511.949951|1511.949951|1497.290039|1502.420044|1502.420044|4274600000|
|2013-02-22|1502.420044|1515.640015|1502.420044|1515.599976|1515.599976|3419

In [37]:
#Add a column
df_sp_500 = df_sp_500.withColumn('IDX', func.lit('SP_500'))

In [38]:
df_sp_500.show()

+----------+-----------+-----------+-----------+-----------+-----------+----------+------+
|      Date|       Open|       High|        Low|      Close|  Adj Close|    Volume|   IDX|
+----------+-----------+-----------+-----------+-----------+-----------+----------+------+
|2013-02-11|1517.930054|1518.310059|1513.609985| 1517.01001| 1517.01001|2684100000|SP_500|
|2013-02-12| 1517.01001|1522.290039|1515.609985|1519.430054|1519.430054|3414370000|SP_500|
|2013-02-13|1519.430054|1524.689941|1515.930054|1520.329956|1520.329956|3385880000|SP_500|
|2013-02-14|1520.329956|1523.140015| 1514.02002|1521.380005|1521.380005|3759740000|SP_500|
|2013-02-15|1521.380005| 1524.23999|1514.140015|1519.790039|1519.790039|3838510000|SP_500|
|2013-02-19|1519.790039|1530.939941|1519.790039|1530.939941|1530.939941|3748910000|SP_500|
|2013-02-20|1530.939941|1530.939941|1511.410034|1511.949951|1511.949951|4240570000|SP_500|
|2013-02-21|1511.949951|1511.949951|1497.290039|1502.420044|1502.420044|4274600000|SP_500|

In [39]:
df_nasdaq = spark.read.csv(bucket.object('nasdaq.csv').uri, header=True, schema=csv_schema).withColumn('IDX', func.lit('NASDAQ'))
df_nasdaq.show()

df_russell_2k = spark.read.csv(bucket.object('russell_2000.csv').uri, header=True, schema=csv_schema).withColumn('IDX', func.lit('RUSSELL_2000'))
df_russell_2k.show()


+----------+-----------+-----------+-----------+-----------+-----------+----------+------+
|      Date|       Open|       High|        Low|      Close|  Adj Close|    Volume|   IDX|
+----------+-----------+-----------+-----------+-----------+-----------+----------+------+
|2013-02-11|3192.530029| 3194.01001|3182.189941|     3192.0|     3192.0|1551370000|NASDAQ|
|2013-02-12| 3190.72998|3196.919922|3184.840088| 3186.48999| 3186.48999|1786800000|NASDAQ|
|2013-02-13|3195.340088| 3205.52002|3187.060059|3196.879883|3196.879883|1822450000|NASDAQ|
|2013-02-14| 3182.73999|3202.330078|3182.389893|3198.659912|3198.659912|1924900000|NASDAQ|
|2013-02-15|3202.840088|3206.209961|3184.030029|3192.030029|3192.030029|1858670000|NASDAQ|
|2013-02-19|3197.459961|3213.600098|3194.919922|3213.590088|3213.590088|1843840000|NASDAQ|
|2013-02-20| 3211.98999|    3213.25|3163.949951|3164.409912|3164.409912|2001800000|NASDAQ|
|2013-02-21|3154.879883|3155.189941|3118.620117| 3131.48999| 3131.48999|2052630000|NASDAQ|

In [40]:
#make a single dataframe with all 3 indicies
df_indices = df_sp_500.unionAll(df_nasdaq).unionAll(df_russell_2k)
#take a sample and show it
df_indices.sample(False, 0.01).show()

+----------+-----------+-----------+-----------+-----------+-----------+----------+------+
|      Date|       Open|       High|        Low|      Close|  Adj Close|    Volume|   IDX|
+----------+-----------+-----------+-----------+-----------+-----------+----------+------+
|2013-12-24| 1828.02002|1833.319946| 1828.02002|1833.319946|1833.319946|1307630000|SP_500|
|2014-03-03|1857.680054|1857.680054|1834.439941| 1845.72998| 1845.72998|3428220000|SP_500|
|2014-03-06|1874.180054|1881.939941|1874.180054|1877.030029|1877.030029|3360450000|SP_500|
|2014-06-24|1961.969971|1968.170044|1948.339966| 1949.97998| 1949.97998|3089700000|SP_500|
|2014-10-31|2001.199951|2018.189941|2001.199951|2018.050049|2018.050049|4292290000|SP_500|
|2015-02-03|2022.709961|2050.300049|2022.709961|2050.030029|2050.030029|4615900000|SP_500|
|2015-03-20|2090.320068|2113.919922|2090.320068|2108.100098|2108.100098|5554120000|SP_500|
|2015-06-12|2107.429932|2107.429932|2091.330078|2094.110107|2094.110107|2719400000|SP_500|

In [41]:
w_by_idx_date = Window.partitionBy('IDX').orderBy("Date")
#Get the previous close
df_indices = df_indices.withColumn('PreviousClose', 
                                   func.lag(df_indices.Close).over(w_by_idx_date))
df_indices.show()
#Compute the daily return
df_indices = df_indices.withColumn('DailyReturn', 
                                   func.when(func.isnull(df_indices.Close / df_indices.PreviousClose), 
                                             0).otherwise(df_indices.Close / df_indices.PreviousClose - 1))
df_indices.show()
w_last_30_days = w_by_idx_date.rowsBetween(-30, 0)
df_indices = df_indices.withColumn('MovingAvg30dayDailyReturn', 
                                   func.avg(df_indices.DailyReturn).over(w_last_30_days))
df_indices.show()

+----------+----------+----------+----------+----------+----------+--------+------------+-------------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|         IDX|PreviousClose|
+----------+----------+----------+----------+----------+----------+--------+------------+-------------+
|2013-02-11|913.549988|913.559998|910.219971|913.030029|913.030029|26841000|RUSSELL_2000|         null|
|2013-02-12| 913.77002|918.169983| 913.72998| 917.52002| 917.52002|34143700|RUSSELL_2000|   913.030029|
|2013-02-13|918.130005|921.289978|916.789978|920.580017|920.580017|33858800|RUSSELL_2000|    917.52002|
|2013-02-14|920.549988|924.679993|918.710022| 923.76001| 923.76001|37597400|RUSSELL_2000|   920.580017|
|2013-02-15|927.330017|927.340027| 921.73999|923.150024|923.150024|38385100|RUSSELL_2000|    923.76001|
|2013-02-19|924.140015|     932.0|924.140015|     932.0|     932.0|37489100|RUSSELL_2000|   923.150024|
|2013-02-20|931.469971|931.650024|     913.5|     913.5|     913

In [42]:
import StringIO
csv = StringIO.StringIO()
df_indices.toPandas().to_csv(csv)
output_csv = bucket.object('output.csv')
output_csv.write_stream(csv.getvalue(), 'text/plain')